In [2]:
import requests
import base64

# Desafío

En este desafío, el servidor permite a un usuario registrarse con un correo electrónico y un conjunto de datos adicionales, y devuelve un perfil cifrado con un algoritmo de cifrado de flujo.

Si el usuario registró el correo usuario@example.com y puso como datos “Juan P. Usuario”, el perfil tendrá la forma:

user=usuario@example.com;data=Juan P. Usuario;role=user

Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma <atributo>=<valor>, y separados por punto y coma.

El servidor devuelve el perfil cifrado, precedido por un nonce de 16 bytes, y codificado en base64.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque los cifradores de flujo son maleables. Es posible modificar el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera predecible.

Este tipo de ataque se denomina bit flipping, porque el cambio de un bit en el texto cifrado provoca un cambio del mismo bit en el texto claro.

Para obtener los textos cifrados, se debe hacer un requerimiento POST a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/stream-bitflip/\<email\>/register
        
dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con dos campos:

- Un campo email, con una dirección de correo a registrar. No es necesario que sea la misma dirección utilizada para acceder al desafío.
- Un campo data, con datos arbitrarios, codificado en base64. Los datos no pueden contener el carácter ‘;’ ni el carácter ‘=’.

La respuesta será un perfil como el descripto, cifrado, concatenado con el nonce y codificado en base64.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar los campos requeridos.

In [3]:
# URL para hacer la solicitud del texto cifrado
email = 'solujan@gmail.com'
data = 'Sofia Lujan'

url = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/stream-bitflip/{email}/register'

payload = {'email':email,'data':base64.b64encode(data.encode('ascii'))}

print(url)
print(payload)

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/stream-bitflip/solujan@gmail.com/register
{'email': 'solujan@gmail.com', 'data': b'U29maWEgTHVqYW4='}


In [4]:
# Solicitud
request = requests.post(url, files=payload)
# Body de la respuesta
message_encoded_in_base64 = request.content
message_encoded_in_base64

b'lLpAyHIX9vcVn0izRPqb+7XDi5fm9XnS/O/5gKv9x0IUhWgpphlS9LTdIroLH4yeMnFB6PKKSTj/ESo2GhEBfnc='

In [5]:
# Decodifico el texto cifrado pues está en base 64 y luego lo convierto a hexadecimal para futuras operaciones xor
message_decoded_from_base64 = base64.b64decode(message_encoded_in_base64)
message_decoded_in_hex = message_decoded_from_base64.hex()
message_decoded_in_hex

'94ba40c87217f6f7159f48b344fa9bfbb5c38b97e6f579d2fceff980abfdc74214856829a61952f4b4dd22ba0b1f8c9e327141e8f28a4938ff112a361a11017e77'

user=solujan@gmail.com;data=Sofia Lujan;role=user

OYGEq+jcveSjfbhYx6r/sBBTK+Syihi0Ua6c85YAA4hJU2Sj1MEKN9bjCd+mun+yMDh+ECraD9pOyupOM9nOEJA=

```
1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16

39 81 84 ab e8 dc bd e4 a3 7d b8 58 c7 aa ff b0


---


10 53 2b e4 b2 8a 18 b4 51 ae 9c f3 96 00 03 88

49 53 64 a3 d4 c1 0a 37 d6 e3 09 df a6 ba 7f b2

30 38 7e 10 2a da 0f da 4e ca ea 4e 33 d9 ce 10

90
```



In [6]:
len(message_decoded_in_hex)
print(len(message_decoded_in_hex[32:]))

98


In [7]:
texto_claro = f'user={email};data={data};role=user'
print(texto_claro)
texto_claro_hex = texto_claro.encode('ascii').hex()
print(texto_claro_hex)

user=solujan@gmail.com;data=Sofia Lujan;role=user
757365723d736f6c756a616e40676d61696c2e636f6d3b646174613d536f666961204c756a616e3b726f6c653d75736572


## C = K ^ P

In [9]:
def xor_two_list(lista_a, lista_b):
  if len(lista_a) != len(lista_b):
    raise ValueError("listas tiene que ser de mismo tamanio")
  new_string = []
  for index in range(0, len(lista_a), 2):
    cn = lista_a[index:index+2]
    cn1 = lista_b[index:index+2]
    cn_xor = int(cn,16) ^ int(cn1,16)
    new_string.append(hex(cn_xor))
  return new_string

In [66]:
noonce = message_decoded_in_hex[:32]
noonce
# 71 41 e8 f2 8a 49 38 ff 11 2a 36 1a 11 01 7e 77

'94ba40c87217f6f7159f48b344fa9bfb'

In [67]:
key = xor_two_list(message_decoded_in_hex[32:], texto_claro_hex)

In [68]:
texto_claro_nuevo = f'user={email};data={data.replace(" ", "")};role=admin'
texto_claro_nuevo

'user=solujan@gmail.com;data=SofiaLujan;role=admin'

In [69]:
texto_claro_nuevo_hex = texto_claro_nuevo.encode('ascii')
texto_claro_nuevo_hex

b'user=solujan@gmail.com;data=SofiaLujan;role=admin'

In [70]:
def xor_two_list_hex(lista_a, lista_b):
  if len(lista_a) != len(lista_b):
    raise ValueError("listas tiene que ser de mismo tamanio")
  new_string = []
  for index in range(0, len(lista_a)):
    cn_xor = lista_a[index] ^ int(lista_b[index],16)
    new_string.append(hex(cn_xor))
  return new_string

In [71]:
result = xor_two_list_hex(texto_claro_nuevo_hex, key)
#94ba40c87217f6f7159f48b344fa9bfbb5c38b97e6f579d2fceff980abfdc74214856829a61952f4b4dd22ba0b1f8c9e327141e8f28a4938ff112a361a11017e77
# -------------------------------b5c38b97e6f579d2fceff980abfdc74214856829a61952f4b4dd22ba0b1f8c9e321d78f7f9851c71e212236e46001f726b

In [72]:
result

['0xb5',
 '0xc3',
 '0x8b',
 '0x97',
 '0xe6',
 '0xf5',
 '0x79',
 '0xd2',
 '0xfc',
 '0xef',
 '0xf9',
 '0x80',
 '0xab',
 '0xfd',
 '0xc7',
 '0x42',
 '0x14',
 '0x85',
 '0x68',
 '0x29',
 '0xa6',
 '0x19',
 '0x52',
 '0xf4',
 '0xb4',
 '0xdd',
 '0x22',
 '0xba',
 '0xb',
 '0x1f',
 '0x8c',
 '0x9e',
 '0x32',
 '0x1d',
 '0x78',
 '0xf7',
 '0xf9',
 '0x85',
 '0x1c',
 '0x71',
 '0xe2',
 '0x12',
 '0x23',
 '0x6e',
 '0x46',
 '0x0',
 '0x1f',
 '0x72',
 '0x6b']

In [73]:
noonce

'94ba40c87217f6f7159f48b344fa9bfb'

In [74]:
result_string = ''
for each in result:
  if len(each[2:]) == 2:
    result_string = result_string + each[2:]
  if len(each[2:]) == 1:
    result_string = result_string + '0' +each[2:]

result_string

'b5c38b97e6f579d2fceff980abfdc74214856829a61952f4b4dd22ba0b1f8c9e321d78f7f9851c71e212236e46001f726b'

In [80]:
total = noonce + result_string

In [81]:
url_answ = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto/stream-bitflip/solujan@gmail.com/answer"
payload = {'message':base64.b64encode(total.encode('ascii'))}


In [82]:
request = requests.post(url_answ, files=payload)


In [83]:
request

<Response [200]>

In [84]:
request.text

'Lo siento, siga participando.\n'